## ECG Heartbeat Categorization

> This dataset is composed of two collections of heartbeat signals derived from two famous datasets in heartbeat classification, the MIT-BIH Arrhythmia Dataset and The PTB Diagnostic ECG Database.

In [ ]:
# importing necessary libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# importing dataset from drive

data = pd.read_csv("../input/heartbeat/mitbih_train.csv", header=None)
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
# showing column wise %ge of NaN values they contains 
null_col = []

for i in df.columns:
  print(i,"\t-\t", df[i].isna().mean()*100)
  if df[i].isna().mean()*100 > 0:
    null_col.append(i)


> Since data does'nt contain any null values, we can move further

In [ ]:
classes = []
sns.countplot(x=187, data = df) 

> Here this bar graph easily shows how data is imbalanced. More than 80% data is in class 0. So, first, we have to balance th data in to get more precise predictions.

> For balancing the data I'm using undersampling in which we will reduce the rows of class 0 to the number compareble to others/

In [ ]:
class_1 = df[df[187]==1.0]
class_2 = df[df[187]==2.0]
class_3 = df[df[187]==3.0]
class_4 = df[df[187]==4.0]
class_0 = df[df[187]==0.0].sample(n = 8000)

In [ ]:
new_df = pd.concat([class_0, class_1, class_2, class_3, class_4])

In [ ]:
new_df.head()

In [ ]:
sns.countplot(x=187, data = new_df) 

> Now for visualising each class, here is plot of any random sample of hearbeat in each class.

In [ ]:
index = 0

fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize=(25,2))

for i in range(5):
  ax[i].plot(new_df[new_df[187]==float(i)].sample(1).iloc[0,:186])
  ax[i].set_title('Class: '+str(i))


In [ ]:
#now lets split data in test train pairs

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new_df.drop([187], axis=1), new_df[187], test_size = 0.1)

In [ ]:
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
from tensorflow.keras import Sequential,utils
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout

In [ ]:
clf = Sequential()

clf.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation='relu', input_shape = (X_train.shape[1],1)))
clf.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation='relu')) 
clf.add(Conv1D(filters=128, kernel_size=(5,), padding='same', activation='relu'))    

clf.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
clf.add(Dropout(0.5))

clf.add(Flatten())

clf.add(Dense(units = 512, activation='relu'))
clf.add(Dense(units = 1024, activation='relu'))

clf.add(Dense(units = 5, activation='softmax'))


In [ ]:
clf.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = clf.fit(X_train, y_train, epochs = 10)

In [ ]:
# Prediction

y_pred = clf.predict(X_test)

In [ ]:
acc = history.history['accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, color='red', label='Training acc')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

y_lbl = [np.where(i == np.max(i))[0][0] for i in y_pred]
mat = confusion_matrix(y_test, y_lbl)
fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(mat, annot = True)

In [ ]:
# Measure the Accuracy Score

from sklearn import metrics

print("Accuracy score of the predictions: {0}".format(metrics.accuracy_score(y_lbl, y_test)))


> As we can see above CNN got accuracy of 96.6%, now we can predict the vaalues for our test dataset.

***

In [ ]:
test_data = pd.read_csv("../input/heartbeat/mitbih_test.csv", header=None)
test_df = pd.DataFrame(test_data)

In [ ]:
test_df.head()

In [ ]:
classes = []
sns.countplot(x=187, data = test_df) 

In [ ]:
index = 0

fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize=(25,2))

for i in range(5):
  ax[i].plot(test_df[test_df[187]==float(i)].sample(1).iloc[0,:186])
  ax[i].set_title('Class: '+str(i))


In [ ]:
test_X = test_df.drop([187], axis=1) 
test_y = test_df[187]

test_X = np.array(test_X).reshape(test_X.shape[0], test_X.shape[1], 1)

In [ ]:
test_pred_y = clf.predict(test_X)

In [ ]:
from sklearn.metrics import confusion_matrix

test_lbl_y = [np.where(i == np.max(i))[0][0] for i in test_pred_y]
mat = confusion_matrix(test_y, test_lbl_y)
fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(mat, annot = True)

In [ ]:
# Measure the Accuracy Score

from sklearn import metrics

print("Accuracy score of the predictions: {0}".format(metrics.accuracy_score(test_lbl_y, test_y)))


> And here our CNN got us 96.5% accuracy on test dataset